In [1]:
import sys, time, os, asyncio, warnings
from datetime import datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient
from lsst.summit.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.extras.rubinTvDatabase import getRubinTvDatabase
# This needs tickets/DM-42894 for summit_extras

In [2]:
client = EfdClient('usdf_efd')
butler = Butler('/repo/embargo', collections="LATISS/raw/all")

# Get the classified mount error data from 2023

In [3]:
infile = open('/home/c/cslage/u/AuxTel/mount_classifier/Mount_Errors_Classified_Dict_29Dec23.pkl', 'rb')
Mount_Errors_Classified_Dict = pkl.load(infile)
infile.close()

# Get the RubinTV json data

In [4]:
json_data = getRubinTvDatabase('LATISS')

# Now walk thru 2023, getting the wind induced failures and the wind speed data

In [5]:
wind_speeds = []
image_degs = []
image_dirs = []
els = []
azs = []
wind_directions = []

startDay = 20230101
endDay = 20231231

dayObs = startDay
while dayObs < endDay:
    # Don't do the dayObs if there is no wind data.
    nextDayObs = calcNextDay(dayObs)
    start = Time(f"{dayObsIntToString(dayObs)} 23:00:00Z", scale='utc')
    end = Time(f"{dayObsIntToString(nextDayObs)} 08:00:00Z", scale='utc')
    maxSpeed = await client.select_time_series('lsst.sal.ESS.airFlow', \
                                                ['maxSpeed'],  start, end, index=301)
    if len(maxSpeed) == 0:
        print(f"No wind data for {dayObs}")
        dayObs = calcNextDay(dayObs)
        continue

    this_rubintv = json_data[json_data['dayObs'] == dayObs]
    seqNums = this_rubintv['seqNum'].values
    counter = 0
    for seqNum in seqNums:
        try:
            if seqNum in Mount_Errors_Classified_Dict[dayObs]['OSC']:
                continue
            if seqNum in Mount_Errors_Classified_Dict[dayObs]['TIM']:
                continue
        except:
            pass
        # Get the image degradation from RubinTV
        seqNum_rubintv = this_rubintv[this_rubintv['seqNum'] == seqNum]
        try:
            image_deg = seqNum_rubintv['Mount motion image degradation'].values[0]
            if np.isnan(image_deg):
                continue
            # Get the other data from the butler
            expId = dayObs * 100000 + seqNum
            dataId = {'detector':0, 'exposure':expId}
            mData = butler.get('raw.metadata', dataId=dataId)
            tStart = mData['DATE-BEG']
            start = Time(tStart, scale='tai').utc
            tEnd = mData['DATE-END']
            end = Time(tEnd, scale='tai').utc
            az = mData['AZSTART']
            el = mData['ELSTART']
            wind_data = await client.select_time_series('lsst.sal.ESS.airFlow', \
                                                    ['maxSpeed','direction'],  start, end, index=301)
            if len(wind_data) == 0:
                # Checks to make sure wind data is there
                continue
        except:
            continue
        wind_speed = np.median(wind_data.values[:,0])
        wind_dir = np.median(wind_data.values[:,1])
        image_dir = (wind_dir - az) 
        # This is the angle between the 
        # wind and the telescope pointing
        els.append(el)
        azs.append(az)
        wind_directions.append(wind_dir)
        image_degs.append(image_deg)
        wind_speeds.append(wind_speed)
        image_dirs.append(image_dir)
        counter += 1
        #print(expId, image_deg, wind_speed, wind_dir, az, image_dir)  
    print(f"Finished {dayObs}, with {counter} good values")
    dayObs = calcNextDay(dayObs)


No wind data for 20230101
No wind data for 20230102
No wind data for 20230103
No wind data for 20230104
No wind data for 20230105
No wind data for 20230106
No wind data for 20230107
No wind data for 20230108
No wind data for 20230109
No wind data for 20230110
No wind data for 20230111
No wind data for 20230112
No wind data for 20230113
No wind data for 20230114
No wind data for 20230115
No wind data for 20230116
No wind data for 20230117
No wind data for 20230118
No wind data for 20230119
No wind data for 20230120
No wind data for 20230121
No wind data for 20230122
No wind data for 20230123
No wind data for 20230124
No wind data for 20230125
No wind data for 20230126
No wind data for 20230127
No wind data for 20230128
No wind data for 20230129
No wind data for 20230130
No wind data for 20230131
No wind data for 20230201
No wind data for 20230202
No wind data for 20230203
No wind data for 20230204
No wind data for 20230205
No wind data for 20230206
No wind data for 20230207
No wind data

In [ ]:
len(wind_speeds)

# Bin the data by wind speed to eliminate the noise

In [ ]:
%matplotlib inline
plot_speeds = []
plot_degs = []
yerr = []
speed_step = 1
for speed in range(0,16,speed_step):
    speed_min = speed
    speed_max = speed + speed_step
    degs = []
    for i in range(len(wind_speeds)):
        if wind_speeds[i] > speed_min and wind_speeds[i] < speed_max:
            deg = image_degs[i]
            if deg < 2.0:
                degs.append(image_degs[i])
    plot_speeds.append(speed + speed_step / 2.0)
    plot_degs.append(np.median(degs))
    yerr.append(np.std(degs) / np.sqrt(len(degs)))
plt.errorbar(plot_speeds, plot_degs, yerr=yerr, marker = 'x')
plt.xlabel("Wind speed (m/s)")
plt.ylabel("Median mount motion image degradation (arcseconds)")
plt.ylim(0,0.5)


# Now try binning by angle

In [ ]:
%matplotlib inline
speed_step = 2
dirs = [[-10, 10, 'blue'], [170, 190, 'red'], [80, 100, 'green']]
for [dir_min, dir_max, color] in dirs:
    plot_speeds = []
    plot_degs = []
    yerr = []

    for speed in range(0,16,speed_step):
        speed_min = speed
        speed_max = speed + speed_step
        degs = []
        for i in range(len(wind_speeds)):
            if wind_speeds[i] > speed_min and wind_speeds[i] < speed_max \
            and ((image_dirs[i] > dir_min and image_dirs[i] < dir_max) \
            or (image_dirs[i] < -dir_min and image_dirs[i] < -dir_max)):
                deg = image_degs[i]
                if deg < 2.0:
                    degs.append(image_degs[i])
        plot_speeds.append(speed + speed_step / 2.0)
        plot_degs.append(np.median(degs))
        yerr.append(np.std(degs) / np.sqrt(len(degs)))
        #print(dir_min, speed_min, len(degs))
    plt.errorbar(plot_speeds, plot_degs, yerr=yerr, marker = 'x', color=color, label=str(dir_min+10))
plt.xlabel("Wind speed (m/s)")
plt.ylabel("Median mount motion image degradation (arcseconds)")
plt.ylim(0,0.5)
plt.legend()


In [ ]:
plt.hist(image_dirs)

In [ ]:
%matplotlib inline
plot_speeds = []
plot_degs = []
for i in range(len(wind_speeds)):
    #if wind_directions[i] > 160.0 and wind_directions[i] < 200.0:
    plot_speeds.append(wind_speeds[i])
    plot_degs.append(image_degs[i])
plt.scatter(plot_speeds, plot_degs)
plt.xlabel("Wind speed (m/s)")
plt.ylabel("Mount motion image degradation (arcseconds)")
plt.ylim(0,1.0)
